In [1]:
import json

# Annotations

In this part, we must understand how the images are labelled to be able to preprocess it.

In [2]:
annot_file = open("raw_data/chessred2k/annotations.json")
annot = json.load(annot_file)

In [3]:
annot.keys()

dict_keys(['info', 'images', 'annotations', 'categories', 'splits'])

In [4]:
[type(annot[key]) for key in annot.keys()]

[dict, list, dict, list, dict]

## Info values

In [5]:
annot['info'].keys()

dict_keys(['description', 'version', 'year', 'date_created', 'author'])

In [6]:
[type(annot['info'][key]) for key in annot['info'].keys()]

[str, str, int, str, str]

In [7]:
for val in annot['info'].values():
    print(val)

Chess Recognition Dataset (ChessReD)
1.0
2023
26/08/2023
Athanasios Masouris


## Images values

In [8]:
first_game = [image for image in annot['images'] if image['game_id'] == 0]
len(first_game)

103

In [9]:
first_game[0]

{'file_name': 'G000_IMG000.jpg',
 'path': 'images/0/G000_IMG000.jpg',
 'camera': 'Huawei P40 pro',
 'height': 3072,
 'width': 3072,
 'game_id': 0,
 'move_id': 0,
 'id': 0}

In [10]:
first_game[102]

{'file_name': 'G000_IMG102.jpg',
 'path': 'images/0/G000_IMG102.jpg',
 'camera': 'Huawei P40 pro',
 'height': 3072,
 'width': 3072,
 'game_id': 0,
 'move_id': 102,
 'id': 102}

In [11]:
annot['images'][103]

{'file_name': 'G001_IMG000.jpg',
 'path': 'images/1/G001_IMG000.jpg',
 'camera': 'Huawei P40 pro',
 'height': 3072,
 'width': 3072,
 'game_id': 1,
 'move_id': 0,
 'id': 103}

Here is the understanding of the image key:
- 'file_name' gives obviously the name of the image
- 'path' provides the location. will be useful to define preprocessing functions with images as arguments
- 'height' and 'width' are quite obvious too.
- 'game_id' is associated to the number of the game. Not that useful at first glance
- 'move_id' won't be very useful either
- 'id' is the position of the image in the whole dataset. To be considered

## Annotations value

In [12]:
annot['annotations'].keys()

dict_keys(['pieces', 'corners'])

In [13]:
[type(annot['annotations'][key]) for key in annot['annotations'].keys()]

[list, list]

In [14]:
[len(annot['annotations'][key]) for key in annot['annotations'].keys()]

[223804, 2078]

In [15]:
annot["annotations"]['pieces'][0]

{'image_id': 0,
 'category_id': 7,
 'chessboard_position': 'a8',
 'id': 0,
 'bbox': [510.34, 963.65, 155.75, 186.14]}

In [16]:
annot["annotations"]['pieces'][1]

{'image_id': 0,
 'category_id': 8,
 'chessboard_position': 'b8',
 'id': 1,
 'bbox': [692.17, 877.29, 154.48, 211.46]}

In [17]:
annot["annotations"]['corners'][0]

{'image_id': 0,
 'corners': {'bottom_right': [2610.3, 1560.9],
  'top_right': [1772.23, 638.59],
  'top_left': [488.7, 1078.7],
  'bottom_left': [1063.3, 2304.1]},
 'id': 223804}

In [18]:
annot["annotations"]['corners'][1]

{'image_id': 1,
 'corners': {'top_right': [892.9, 931.7],
  'bottom_right': [2376.27, 994.8],
  'bottom_left': [2392.7, 2142.33],
  'top_left': [429.86, 1982.92]},
 'id': 223805}

## Categories value.

In [19]:
annot['categories']

[{'id': 0, 'name': 'white-pawn'},
 {'id': 1, 'name': 'white-rook'},
 {'id': 2, 'name': 'white-knight'},
 {'id': 3, 'name': 'white-bishop'},
 {'id': 4, 'name': 'white-queen'},
 {'id': 5, 'name': 'white-king'},
 {'id': 6, 'name': 'black-pawn'},
 {'id': 7, 'name': 'black-rook'},
 {'id': 8, 'name': 'black-knight'},
 {'id': 9, 'name': 'black-bishop'},
 {'id': 10, 'name': 'black-queen'},
 {'id': 11, 'name': 'black-king'},
 {'id': 12, 'name': 'empty'}]

## Splits value

In [20]:
annot['splits'].keys()

dict_keys(['train', 'val', 'test', 'chessred2k'])

In [21]:
annot['splits']['chessred2k']['train'].keys()

dict_keys(['image_ids', 'n_samples'])

## Creating board Y-value

In [24]:
len(annot['images'])

10800

In [25]:
pieces_game_zero = [annot["annotations"]['pieces'][i] for i in range(223804) if annot["annotations"]['pieces'][i]['image_id'] == 0]

In [26]:
len(pieces_game_zero)

32

In [30]:
import numpy as np

In [37]:
def array_board(pieces_game:list):
    res_array = np.ones((8,8))*12
    letter_conv = {'a':0, 'b':1, 'c':2, 'd':3, 'e':4, 'f':5, 'g':6, 'h':7}
    for piece in pieces_game:
        pos = piece['chessboard_position']
        row, col = 8-int(pos[1]), letter_conv[pos[0]]
        res_array[row, col] = piece['category_id']
    return res_array

In [38]:
array_board(pieces_game_zero)

array([[ 7.,  8.,  9., 10., 11.,  9.,  8.,  7.],
       [ 6.,  6.,  6.,  6.,  6.,  6.,  6.,  6.],
       [12., 12., 12., 12., 12., 12., 12., 12.],
       [12., 12., 12., 12., 12., 12., 12., 12.],
       [12., 12., 12., 12., 12., 12., 12., 12.],
       [12., 12., 12., 12., 12., 12., 12., 12.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 1.,  2.,  3.,  4.,  5.,  3.,  2.,  1.]])

In [40]:
fen_str = "PRNBQKprnbqk"
id_range = [i for i in range(12)]
dict_fen = dict([(k, fen_str[k]) for k in id_range])
dict_fen

{0: 'P',
 1: 'R',
 2: 'N',
 3: 'B',
 4: 'Q',
 5: 'K',
 6: 'p',
 7: 'r',
 8: 'n',
 9: 'b',
 10: 'q',
 11: 'k'}